In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ogw3-train-test/test_dataset.npz
/kaggle/input/ogw3-train-test/train_dataset.npz
/kaggle/input/ogw3-augmented/X_damage_augmented.npy
/kaggle/input/ogw3-augmented/X_freq_augmented.npy
/kaggle/input/ogw3-augmented/X_signals_augmented.npy
/kaggle/input/ogw3-loaded/data_frames_ogw3.pkl


# 1.5) loading augmented data

In [2]:
import os
import matplotlib.pyplot as plt

In [3]:
%pwd


'/kaggle/working'

In [4]:
os.chdir("..")
os.chdir("input")

In [5]:
X_signals_augmented = np.load('ogw3-augmented/X_signals_augmented.npy')
X_damage_augmented = np.load('ogw3-augmented/X_damage_augmented.npy')
X_freq_augmented = np.load('ogw3-augmented/X_freq_augmented.npy')


In [6]:
print(f"X_signals shape:{X_signals_augmented.shape}")
print(f"X_signals shape:{X_damage_augmented.shape}")
print(f"X_signals shape:{X_freq_augmented.shape}")

X_signals shape:(2040, 4369, 1)
X_signals shape:(2040, 14)
X_signals shape:(2040, 12)


In [7]:
import numpy as np

def summarize_data(X, name):
    print(f"Summary for {name}:")
    print("Mean:", np.mean(X, axis=0))
    print("Variance:", np.var(X, axis=0))
    print("Min:", np.min(X, axis=0))
    print("Max:", np.max(X, axis=0))
    print("-" * 50)

summarize_data(X_signals_augmented, "X_signals_augmented")
summarize_data(X_freq_augmented, "X_freq_augmented")


Summary for X_signals_augmented:
Mean: [[-0.00808563]
 [-0.03676917]
 [ 0.00793156]
 ...
 [ 0.43494834]
 [ 0.47826858]
 [ 0.47794448]]
Variance: [[0.04756828]
 [0.05728288]
 [0.05182533]
 ...
 [0.11266843]
 [0.11778735]
 [0.1192308 ]]
Min: [[-1.]
 [-1.]
 [-1.]
 ...
 [-1.]
 [-1.]
 [-1.]]
Max: [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
--------------------------------------------------
Summary for X_freq_augmented:
Mean: [0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333]
Variance: [0.07638889 0.07638889 0.07638889 0.07638889 0.07638889 0.07638889
 0.07638889 0.07638889 0.07638889 0.07638889 0.07638889 0.07638889]
Min: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Max: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
--------------------------------------------------


In [8]:
import pandas as pd

damage_counts = pd.Series(X_damage_augmented.flatten()).value_counts()
print("Damage Condition Counts:\n", damage_counts)


Damage Condition Counts:
 0.0    26520
1.0     2040
Name: count, dtype: int64


# 2. Execution and model Training
This is where the actual execution starts

In [9]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import os

In [10]:
os.chdir("..")
os.chdir("working")

In [11]:
os.chdir("..")
%pwd

'/kaggle'

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split

# ----------------------------------------------------------------------
# Step 1: Convert one-hot encoded damage labels to scalar values.
# ----------------------------------------------------------------------
# Each sample in X_damage_augmented is a one-hot vector. We convert it to the
# corresponding class index (i.e., 0 for undamaged, 1-13 for damage levels).
y_augmented = np.argmax(X_damage_augmented, axis=1).astype(np.float32)
# y_augmented now has shape (num_samples,)

In [13]:
y_augmented.min()

0.0

In [14]:
# ----------------------------------------------------------------------
# Step 2: Define the features.
# ----------------------------------------------------------------------
# For this regression task, we use the guided wave signals as the features.
# (If you wish to include frequency information as additional features, you might
# concatenate or process X_freq_augmented separately; here we assume only the signal is used.)
X_augmented = X_signals_augmented  # shape: (num_samples, 4369, 1)

In [15]:
# ----------------------------------------------------------------------
# Step 3: Split the data into training and test sets.
# ----------------------------------------------------------------------
# Here we use 80% of the data for training and 20% for testing.
X_train, X_test, y_train, y_test = train_test_split(
    X_augmented, y_augmented, test_size=0.1, random_state=42
)


In [16]:



# ----------------------------------------------------------------------
# Step 4: Verify the shapes of the datasets.
# ----------------------------------------------------------------------
print(f"Training data shape: {X_train.shape}")  # Expected: (num_train_samples, 4369, 1)
print(f"Training labels shape: {y_train.shape}")  # Expected: (num_train_samples,)
print(f"Test data shape: {X_test.shape}")         # Expected: (num_test_samples, 4369, 1)
print(f"Test labels shape: {y_test.shape}")         # Expected: (num_test_samples,)


Training data shape: (1836, 4369, 1)
Training labels shape: (1836,)
Test data shape: (204, 4369, 1)
Test labels shape: (204,)


In [17]:

# Identify indices where labels are not zero
non_zero_indices = y_train != 0

# Filter training data and labels
X_train = X_train[non_zero_indices]
y_train = y_train[non_zero_indices]

# Identify indices where test labels are not zero
non_zero_indices_test = y_test != 0

# Filter test data and labels
X_test = X_test[non_zero_indices_test]
y_test = y_test[non_zero_indices_test]

# Verify the new shapes
print(f"Filtered Training data shape: {X_train.shape}")
print(f"Filtered Training labels shape: {y_train.shape}")
print(f"Filtered Test data shape: {X_test.shape}")
print(f"Filtered Test labels shape: {y_test.shape}")


Filtered Training data shape: (1403, 4369, 1)
Filtered Training labels shape: (1403,)
Filtered Test data shape: (157, 4369, 1)
Filtered Test labels shape: (157,)


In [18]:
def print_label_distribution(y, set_name):
    """Print the distribution of labels in a dataset."""
    print(f"\n{set_name} label distribution:")
    print(f"- Baseline samples (0): {np.sum(y == 0)}")
    for damage_size in range(1, 14):
        print(f"- Damage size {damage_size}: {np.sum(y == damage_size)}")

In [19]:
# Print label distribution for training and test sets
print_label_distribution(y_train, "Training set")
print_label_distribution(y_test, "Test set")


Training set label distribution:
- Baseline samples (0): 0
- Damage size 1: 107
- Damage size 2: 107
- Damage size 3: 105
- Damage size 4: 106
- Damage size 5: 105
- Damage size 6: 107
- Damage size 7: 115
- Damage size 8: 109
- Damage size 9: 107
- Damage size 10: 111
- Damage size 11: 109
- Damage size 12: 111
- Damage size 13: 104

Test set label distribution:
- Baseline samples (0): 0
- Damage size 1: 13
- Damage size 2: 13
- Damage size 3: 15
- Damage size 4: 14
- Damage size 5: 15
- Damage size 6: 13
- Damage size 7: 5
- Damage size 8: 11
- Damage size 9: 13
- Damage size 10: 9
- Damage size 11: 11
- Damage size 12: 9
- Damage size 13: 16


In [20]:
# Convert input data to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [21]:
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [22]:
print("Min values after scaling:", np.min(X_train, axis=(1,2)))
print("Max values after scaling:", np.max(X_train, axis=(1,2)))

Min values after scaling: [-1.040155  -0.9812768 -1.        ... -0.918774  -0.9722817 -0.9399686]
Max values after scaling: [1.0318567  0.94322634 1.         ... 0.9836663  0.9312466  1.0010613 ]


In [23]:
print("X_train dtype:", X_train.dtype)  # Should print float32
print("y_train dtype:", y_train.dtype)  # Should print float32

X_train dtype: float32
y_train dtype: float32


In [24]:
import numpy as np

# Compute the minimum and maximum values from the training targets.
y_min = np.min(y_train)
y_max = np.max(y_train)

# Normalize the training and test targets.
y_train_norm = (y_train - y_min) / (y_max - y_min)
y_test_norm = (y_test - y_min) / (y_max - y_min)


print("Before Normalizing training targets (first 5):", y_train[:5])
print("Before Normalizing test targets (first 5):", y_test[:5])

print("Normalized training targets (first 5):", y_train_norm[:5])
print("Normalized test targets (first 5):", y_test_norm[:5])


Before Normalizing training targets (first 5): [ 5.  3. 13.  1. 10.]
Before Normalizing test targets (first 5): [7. 6. 1. 8. 4.]
Normalized training targets (first 5): [0.33333334 0.16666667 1.         0.         0.75      ]
Normalized test targets (first 5): [0.5        0.41666666 0.         0.5833333  0.25      ]


# 3. Model Architecture

In [25]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [26]:
tf.keras.backend.set_floatx('float32')

In [27]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [28]:
# Multi-GPU setup
strategy = tf.distribute.MirroredStrategy()

In [29]:
# Transformer Encoder block definition remains unchanged
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res


In [30]:
# Model building function adapted for regression
def build_regression_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    # Reshape the input to 3D (batch_size, 874, 1)
    x = layers.Reshape((input_shape[0], 1))(inputs)
    
    # Apply multiple Transformer Encoder blocks
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    # Flatten the tensor for the MLP part
    x = layers.Flatten()(x)
    
    # MLP layers
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    
    # Output layer for regression: Predict a single value.
    # Using a sigmoid activation and scaling by 13 ensures outputs in [0, 13].
    dense = layers.Dense(1, activation="sigmoid")(x)
    outputs = layers.Lambda(lambda x: 13 * x)(dense)
    
    return keras.Model(inputs, outputs)


In [31]:
# Multi-GPU strategy scope
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    input_shape = (874,)  # Adjusted input shape

    model = build_regression_model(
        input_shape,
        head_size=512,
        num_heads=8,
        ff_dim=128,
        num_transformer_blocks=4,
        mlp_units=[256],
        mlp_dropout=0.4,
        dropout=0.3,
    )

    # Compile with a regression loss (mean squared error) and a regression metric (MAE)
    model.compile(
        loss="mse",  # Mean Squared Error loss for regression
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        metrics=["mae"],  # Mean Absolute Error as an additional metric
    )

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 874)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 874, 1)         │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 874, 1)         │         28,673 │ reshape[0][0],         │
│ (MultiHeadAttention)      │                        │                │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 874, 1)         │              0 │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 874, 1)         │              2 │ dropout_1[0][0]        │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 874, 1)         │              0 │ layer_normalization[0… │
│                           │                        │                │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 874, 128)       │            256 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 874, 128)       │              0 │ conv1d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 874, 1)         │            129 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 874, 1)         │              2 │ conv1d_1[0][0]         │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 874, 1)         │              0 │ layer_normalization_1… │
│                           │                        │                │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 874, 1)         │         28,673 │ add_1[0][0],           │
│ (MultiHeadAttention)      │                        │                │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 874, 1)         │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 874, 1)         │              2 │ dropout_4[0][0]        │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 874, 1)         │              0 │ layer_normalization_2… │
│                           │                        │                │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)    

 Total params: 340,505 (1.30 MB)

 Trainable params: 340,505 (1.30 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(
    filepath='model_checkpoint.keras',  # Save file with .keras extension
    save_weights_only=False,  # Save the full model
    save_freq='epoch',  # Save at the end of each epoch
    verbose=1  # To print a message when the model is saved
)

# Modify the callbacks list to include the checkpoint callback 
callbacks = [
    keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True),
    checkpoint_callback
]

In [33]:
X_train.shape

(1403, 4369, 1)

In [34]:
y_train_norm.shape

(1403,)

In [35]:
y_train_norm,y_test_norm

(array([0.33333334, 0.16666667, 1.        , ..., 0.75      , 0.25      ,
        0.41666666], dtype=float32),
 array([0.5       , 0.41666666, 0.        , 0.5833333 , 0.25      ,
        0.25      , 0.        , 0.25      , 0.6666667 , 0.6666667 ,
        0.9166667 , 0.16666667, 1.        , 0.33333334, 0.33333334,
        0.08333334, 0.6666667 , 0.9166667 , 0.5833333 , 0.16666667,
        0.25      , 0.        , 0.16666667, 0.5       , 0.33333334,
        0.33333334, 0.6666667 , 0.75      , 0.75      , 0.25      ,
        0.16666667, 0.33333334, 0.75      , 1.        , 0.9166667 ,
        0.5       , 0.08333334, 0.8333333 , 0.41666666, 0.41666666,
        0.9166667 , 0.25      , 0.16666667, 0.25      , 0.41666666,
        0.6666667 , 0.41666666, 0.5833333 , 0.33333334, 0.41666666,
        0.9166667 , 0.33333334, 0.16666667, 1.        , 0.        ,
        0.6666667 , 0.8333333 , 0.41666666, 0.41666666, 0.6666667 ,
        0.5833333 , 0.25      , 0.33333334, 1.        , 0.9166667 ,
      

In [36]:
# Assume X_train has the shape (num_samples, 4369, 1)
num_samples, original_length, channels = X_train.shape

# Calculate padding length so that the length becomes divisible by 5.
pad_len = 1

# Pad along the sequence axis (axis=1) with -np.inf so that the max operation is not affected.
X_train_padded = np.pad(
    X_train,
    pad_width=((0, 0), (0, pad_len), (0, 0)),
    mode='constant',
    constant_values=-np.inf
)

# Now X_train_padded has the shape (num_samples, 4370, 1).


In [37]:
X_train_padded.shape

(1403, 4370, 1)

In [38]:

# Reshape to group every 5 consecutive elements:
# New shape will be (num_samples, 874, 5, 1)
grouped_length = X_train_padded.shape[1] // 5  # Should be 874
X_train_grouped = X_train_padded.reshape(num_samples, grouped_length, 5, channels)

# Take the max over the 5 elements for each group (axis=2)
# Resulting shape is (num_samples, 874, 1)
X_train_max = np.max(X_train_grouped, axis=2)

# Optionally, if your model expects inputs of shape (num_samples, 874),
# reshape the array from (num_samples, 874, 1) to (num_samples, 874)
X_train_final = X_train_max.reshape(num_samples, grouped_length)

# Check the resulting shape
print("Shape after max-pooling and reshaping:", X_train_final.shape)


Shape after max-pooling and reshaping: (1403, 874)


In [39]:
# Assume X_train has the shape (num_samples, 4369, 1)
num_samples, original_length, channels = X_test.shape

# Calculate padding length so that the length becomes divisible by 5.
pad_len = 1

# Pad along the sequence axis (axis=1) with -np.inf so that the max operation is not affected.
X_test_padded = np.pad(
    X_test,
    pad_width=((0, 0), (0, pad_len), (0, 0)),
    mode='constant',
    constant_values=-np.inf
)

# Now X_train_padded has the shape (num_samples, 4370, 1).

# Reshape to group every 5 consecutive elements:
# New shape will be (num_samples, 874, 5, 1)
grouped_length = X_test_padded.shape[1] // 5  # Should be 874
X_test_grouped = X_test_padded.reshape(num_samples, grouped_length, 5, channels)

# Take the max over the 5 elements for each group (axis=2)
# Resulting shape is (num_samples, 874, 1)
X_test_max = np.max(X_test_grouped, axis=2)

# Optionally, if your model expects inputs of shape (num_samples, 874),
# reshape the array from (num_samples, 874, 1) to (num_samples, 874)
X_test_final = X_test_max.reshape(num_samples, grouped_length)

# Check the resulting shape
print("Shape after max-pooling and reshaping:", X_test_final.shape)


Shape after max-pooling and reshaping: (157, 874)


In [40]:
%pwd

'/kaggle'

In [41]:
os.chdir("working")

In [42]:
# Train the model.
# Note: y_train and y_test should now contain continuous values between 0.0 and 13.0.
history = model.fit(
    X_train_final,
    y_train,
    validation_data=(X_test_final, y_test),
    epochs=300,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/300
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 16.3684 - mae: 3.3873
Epoch 1: saving model to model_checkpoint.keras
44/44 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - loss: 16.3592 - mae: 3.3867 - val_loss: 13.1051 - val_mae: 3.1299
Epoch 2/300
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 15.5687 - mae: 3.3198
Epoch 2: saving model to model_checkpoint.keras
44/44 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - loss: 15.5461 - mae: 3.3170 - val_loss: 12.7784 - val_mae: 3.0390
Epoch 3/300
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 13.6669 - mae: 3.0571
Epoch 3: saving model to model_checkpoint.keras
44/44 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - loss: 13.6710 - mae: 3.0581 - val_loss: 12.2695 - val_mae: 2.9300
Epoch 4/300
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 12.5890 - mae: 2.9630
Epoch 4: saving model to model_checkpoint.keras
44/44 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - loss: 12.5896 - mae: 2.9630 - val_loss: 12.4007 - val_mae: 2.9575
Epoch 5/300
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 12